# Investment crowdfunding has little faith in sustainability! At least for the moment.

## Journal:

*Venture Capital*, 25:1, 91-115, 2023, [DOI: 10.1080/13691066.2022.2129510](https://doi.org/10.1080/13691066.2022.2129510)

## Authors:

Carmen Mendoza

Isabel María Parra Oller

Álvaro Rezola (@alvarorezola)

Nuria Suárez

### Table 1: Descriptive statistics
This table shows the descriptive statistics – mean, standard deviation, 25th percentile, median, 75th percentile – of the main variables of interest (columns (1) to (7)). Columns (8) to (10) show the mean values of the main variables across the two subsamples of offerings and the T-statistic for the mean differences. The T-statistics reported are obtained for the differences between the means across groups of offerings. All the variables are defined in Annex 1. ***, **, and * indicate statistical significance at 1, 5, and 10 percent, respectively.

### Table 2: Propensity Score Matching 
This table shows the mean values of the main variables across the two subsamples of offerings and the t-statistics obtained for the differences between the means across groups of offerings, before matching and after matching using caliper, nearest 1-to-1 and nn-VBC methods. ***, ** and * indicate statistical significance at 1, 5, and 10%, respectively.

### Table 3: Average Treatment Effect on the Treated (ATET) 
This table shows the average treatment effect on the treated individuals (ATET) for each method: caliper, nearest 1-to-1 and nn-VBC methods. ***, ** and * indicate statistical significance at 1, 5, and 10%, respectively.

### Table 4: Sustainability and success
This table presents IV results examining the effect of the sustainable orientation of investment crowdfunding offerings on the probability of success. The dependent variable in columns (1) and (2) is the dummy that identifies sustainable offerings (Sustainable). The dependent variable in columns (3) and (5) is SUCCESS. QUICK75 is the dependent variables in columns (4) and (6). Variables definitions are reported in Annex 1. Year, industry-year and state-year fixed effects are included but not reported. T-statistics are in parentheses. ***, ** and * indicate statistical significance at 1, 5, and 10%, respectively.

### Table 5: Sustainability and success: the role of firm- and offering-level characteristics
This table presents results examining the effect of firm- and offering-level characteristics on the relationship between the sustainable orientation of investment crowdfunding offerings and the probability of success. The dependent variable is SUCCESS. Variables definitions are reported in Annex 1. Year, industry-year and state-year fixed effects are included but not reported. T-statistics are in parentheses. *** and ** indicate statistical significance at 1 and 5 percent, respectively.

### Table 6: Sustainability and success: the role of the financing environment
This table presents results examining the effect of the characteristics of the financing environment on the relationship between the sustainable orientation of investment crowdfunding offerings and the probability of success. The dependent variable is SUCCESS. Variables definitions are reported in Annex 1. Firm and offering control variables, year, industry-year and state-year fixed effects are included but not reported. T-statistics are in parentheses. ***; ** and * indicate statistical significance at 1, 5, and 10 percent, respectively.

### Table 7: Sustainability and success: robustness tests
This table presents a set of robustness tests for the relationship between the sustainable orientation of investment crowdfunding offerings and the probability of success. In column (1), we report the results for the second-stage regression for the Heckman (1979) method. In column (2), we find that the results do not vary when controlling for the characteristics of the team in terms of gender and size. In columns (3) to (5), we control for the funding history of the company. In column (6), we control for the cost structure defined by the funding portal. The dependent variable is SUCCESS. Variables definitions are reported in Annex 1. Firm and offering control variables, year, industry-year and state-year fixed effects are included but not reported. T-statistics are in parentheses. ***; ** and * indicate statistical significance at 1, 5, and 10 percent, respectively.